# 📈 Прогнозирование LTV клиента (Lifetime Value— пожизненная ценность )

**Цель проекта** — построение модели машинного обучения, способной предсказывать пожизненную ценность клиента (LTV) на основе транзакционных и поведенческих данных. Это позволит:

- Сегментировать клиентов по прибыльности.<p>
- Оптимизировать маркетинговые затраты.<p>
- Повысить точность прогнозов выручки.<p>

**🔍Описание проекта**<p>
Набор данных содержит информацию о 100 тысячах реальных заказов, сделанных с 2016 по 2018 год на нескольких торговых площадках Бразилии: от статуса заказа, цены, оплаты и доставки до местоположения клиента, характеристик товара и, наконец, отзывов покупателей.

**данные:** [Brazilian E-Commerce Public Dataset by Olist](https://www.kaggle.com/datasets/olistbr/brazilian-ecommerce) 


<u>olist_customers_dataset.csv</u> -набор данных содержит информацию о клиенте и его местоположении.<p> 
<u>olist_geolocation_dataset.csv</u>  -набор данных содержит информацию о почтовых индексах Бразилии и её координатах (широта и долгота).<p> 
<u>olist_order_items_dataset.csv</u> -набор данных включает данные о товарах, купленных в каждом заказе.<p>
<u>olist_order_payments_dataset.csv</u> -набор данных включает данные о вариантах оплаты заказов.<p>
<u>olist_order_reviews_dataset.csv</u> -набор данных включает данные об отзывах покупателей.<p>
<u>olist_orders_dataset.csv</u> -основной набор данных. В каждом заказе вы можете найти всю остальную информацию.<p>
<u>olist_products_dataset.csv</u> -набор данных включает данные о продуктах, продаваемых Olist.<p>
<u>olist_sellers_dataset.csv</u> -набор данных содержит информацию о продавцах, которые выполнили заказы, сделанные в Olist.<p>
<u>product_category_name_translation.csv</u> -переводит название категории продукта на английский язык.<p>

# Load Data

In [8]:
import pandas as pd
import numpy as np
import os
import shutil
import kagglehub


In [9]:
# Папка назначения

DATA_DIR = os.path.abspath("../data/raw")
os.makedirs(DATA_DIR, exist_ok=True)

# Скачиваем с Kaggle (olistbr/brazilian-ecommerce)

print("Скачиваем датасет с Kaggle...")
kaggle_path = kagglehub.dataset_download("olistbr/brazilian-ecommerce")

# Перемещаем файлы

print("Перемещаем датасеты...")
for filename in os.listdir(kaggle_path):
    src_file = os.path.join(kaggle_path, filename)
    dst_file = os.path.join(DATA_DIR, filename)

Скачиваем датасет с Kaggle...
Перемещаем датасеты...


In [10]:
# Пути

RAW_PATH = 'data/raw'
PROCESSED_PATH = 'data/processed'

# Создание processed-папки

os.makedirs(PROCESSED_PATH, exist_ok=True)

In [11]:
# Загрузка всех датасетов

customers = pd.read_csv(os.path.join(RAW_PATH, 'olist_customers_dataset.csv'))

orders = pd.read_csv(os.path.join(RAW_PATH, 'olist_orders_dataset.csv'))

order_items = pd.read_csv(
    os.path.join(RAW_PATH, 'olist_order_items_dataset.csv'))

products = pd.read_csv(os.path.join(RAW_PATH, 'olist_products_dataset.csv'))

payments = pd.read_csv(
    os.path.join(RAW_PATH, 'olist_order_payments_dataset.csv'))

reviews = pd.read_csv(os.path.join(RAW_PATH,
                                   'olist_order_reviews_dataset.csv'))

sellers = pd.read_csv(os.path.join(RAW_PATH, 'olist_sellers_dataset.csv'))

categories = pd.read_csv(
    os.path.join(RAW_PATH, 'product_category_name_translation.csv'))

# Preprocessed data

In [12]:
# Приведение столбцов orders с датами к datetime

date_cols_orders = [
    'order_purchase_timestamp', 'order_approved_at',
    'order_delivered_carrier_date', 'order_delivered_customer_date',
    'order_estimated_delivery_date'
]
for col in date_cols_orders:
    orders[col] = pd.to_datetime(orders[col], errors='coerce')

In [13]:
order_items['shipping_limit_date'] = pd.to_datetime(
    order_items['shipping_limit_date'], errors='coerce')
reviews['review_creation_date'] = pd.to_datetime(
    reviews['review_creation_date'], errors='coerce')
reviews['review_answer_timestamp'] = pd.to_datetime(
    reviews['review_answer_timestamp'], errors='coerce')

In [14]:
# Заполнение пропусков в reviews

reviews['review_comment_title'] = reviews['review_comment_title'].fillna(
    reviews['review_comment_title'].mode()[0])
reviews['review_comment_message'] = reviews['review_comment_message'].fillna(
    reviews['review_comment_message'].mode()[0])

In [15]:
# Удаление дубликатов

dfs = [customers, orders, order_items, payments, products, reviews, sellers, categories]
dfs_names = ['customers', 'orders', 'order_items', 'payments', 'products', 'reviews', 'sellers', 'categories']

for df, name in zip(dfs, dfs_names):
    dup_count = df.duplicated().sum()
    print(f"Дубликаты в {name}: {dup_count}")
    df.drop_duplicates(inplace=True)



Дубликаты в customers: 0
Дубликаты в orders: 0
Дубликаты в order_items: 0
Дубликаты в payments: 0
Дубликаты в products: 0
Дубликаты в reviews: 0
Дубликаты в sellers: 0
Дубликаты в categories: 0


In [16]:
# Проверка и анализ пропусков

for df, name in zip(dfs, dfs_names):
    missing = df.isnull().sum()
    missing = missing[missing > 0]
    print(f"Пропуски в {name}:\n{missing}\n")

Пропуски в customers:
Series([], dtype: int64)

Пропуски в orders:
order_approved_at                 160
order_delivered_carrier_date     1783
order_delivered_customer_date    2965
dtype: int64

Пропуски в order_items:
Series([], dtype: int64)

Пропуски в payments:
Series([], dtype: int64)

Пропуски в products:
product_category_name         610
product_name_lenght           610
product_description_lenght    610
product_photos_qty            610
product_weight_g                2
product_length_cm               2
product_height_cm               2
product_width_cm                2
dtype: int64

Пропуски в reviews:
Series([], dtype: int64)

Пропуски в sellers:
Series([], dtype: int64)

Пропуски в categories:
Series([], dtype: int64)



In [17]:
# Заполнение пропусков в products

products['product_category_name'] = products['product_category_name'].fillna('unknown')
products['product_name_lenght'] = products['product_name_lenght'].fillna(products['product_name_lenght'].median())
products['product_description_lenght'] = products['product_description_lenght'].fillna(products['product_description_lenght'].median())
products['product_photos_qty'] = products['product_photos_qty'].fillna(products['product_photos_qty'].median())

for col in ['product_weight_g', 'product_length_cm', 'product_height_cm', 'product_width_cm']:
    products[col] = products[col].fillna(products[col].median())

#Заполнение пропусков в reviews

reviews['review_comment_title'] = reviews['review_comment_title'].fillna(reviews['review_comment_title'].mode()[0])
reviews['review_comment_message'] = reviews['review_comment_message'].fillna(reviews['review_comment_message'].mode()[0])


In [18]:
#Если заказ не одобрен, то заменим на время покупки

orders['order_approved_at'] = orders['order_approved_at'].fillna(
    orders['order_purchase_timestamp'])

# Отсутствует дата передачи курьеру — заполним медианой отгрузки

orders['order_delivered_carrier_date'] = orders[
    'order_delivered_carrier_date'].fillna(
        orders['order_approved_at'].median())

# Отсутствует дата доставки клиенту — заменим медианой среди доставленных

orders['order_delivered_customer_date'] = orders[
    'order_delivered_customer_date'].fillna(
        orders['order_delivered_customer_date'].median())

In [19]:
# Фильтрация "мусорных" заказов (например, цена < 5₽)

order_items = order_items[order_items['price'] >= 5]



In [20]:
# Объединение

orders_customers = pd.merge(orders, customers, on='customer_id', how='inner')
orders_items = pd.merge(orders_customers, order_items, on='order_id', how='inner')
orders_items_products = pd.merge(orders_items, products, on='product_id', how='left')
orders_items_products = pd.merge(orders_items_products, categories, on='product_category_name', how='left')
orders_full = pd.merge(orders_items_products, payments, on='order_id', how='left')
orders_full = pd.merge(orders_full, reviews, on='order_id', how='left')
orders_full = pd.merge(orders_full, sellers, on='seller_id', how='left')



In [21]:
orders_full

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,customer_unique_id,customer_zip_code_prefix,...,payment_value,review_id,review_score,review_comment_title,review_comment_message,review_creation_date,review_answer_timestamp,seller_zip_code_prefix,seller_city,seller_state
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18,7c396fd4830fd04220f754e42b4e5bff,3149,...,18.12,a54f0611adc9ed256b57ede6b6eb5114,4.0,Recomendo,"Não testei o produto ainda, mas ele veio corre...",2017-10-11,2017-10-12 03:43:48,9350,maua,SP
1,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18,7c396fd4830fd04220f754e42b4e5bff,3149,...,2.00,a54f0611adc9ed256b57ede6b6eb5114,4.0,Recomendo,"Não testei o produto ainda, mas ele veio corre...",2017-10-11,2017-10-12 03:43:48,9350,maua,SP
2,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18,7c396fd4830fd04220f754e42b4e5bff,3149,...,18.59,a54f0611adc9ed256b57ede6b6eb5114,4.0,Recomendo,"Não testei o produto ainda, mas ele veio corre...",2017-10-11,2017-10-12 03:43:48,9350,maua,SP
3,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13,af07308b275d755c9edb36a90c618231,47813,...,141.46,8d5266042046a06655c8db133d120ba5,4.0,Muito boa a loja,Muito bom o produto.,2018-08-08,2018-08-08 18:37:50,31570,belo horizonte,SP
4,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04,3a653a41f6f9fc3d2a113cf8398680e8,75265,...,179.12,e73b67b67587f7644d5bd1a52deb1b01,5.0,Recomendo,Muito bom,2018-08-18,2018-08-22 19:07:58,14840,guariba,SP
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
118180,63943bddc261676b46f01ca7ac2f7bd8,1fca14ff2861355f6e5f14306ff977a7,delivered,2018-02-06 12:58:58,2018-02-06 13:10:37,2018-02-07 23:22:42,2018-02-28 17:37:56,2018-03-02,da62f9e57a76d978d02ab5362c509660,11722,...,195.00,29bb71b2760d0f876dfa178a76bc4734,4.0,Recomendo,So uma peça que veio rachado mas tudo bem rs,2018-03-01,2018-03-02 17:50:01,17602,tupa,SP
118181,83c1379a015df1e13d02aae0204711ab,1aa71eb042121263aafbe80c1b562c9c,delivered,2017-08-27 14:46:43,2017-08-27 15:04:16,2017-08-28 20:52:26,2017-09-21 11:24:17,2017-09-27,737520a9aad80b3fbbdad19b66b37b30,45920,...,271.01,371579771219f6db2d830d50805977bb,5.0,Recomendo,Foi entregue antes do prazo.,2017-09-22,2017-09-22 23:10:57,8290,sao paulo,SP
118182,11c177c8e97725db2631073c19f07b62,b331b74b18dc79bcdf6532d51e1637c1,delivered,2018-01-08 21:28:27,2018-01-08 21:36:21,2018-01-12 15:35:03,2018-01-25 23:32:54,2018-02-15,5097a5312c8b157bb7be58ae360ef43c,28685,...,441.16,8ab6855b9fe9b812cd03a480a25058a1,2.0,Recomendo,Foi entregue somente 1. Quero saber do outro p...,2018-01-26,2018-01-27 09:16:56,37175,ilicinea,MG
118183,11c177c8e97725db2631073c19f07b62,b331b74b18dc79bcdf6532d51e1637c1,delivered,2018-01-08 21:28:27,2018-01-08 21:36:21,2018-01-12 15:35:03,2018-01-25 23:32:54,2018-02-15,5097a5312c8b157bb7be58ae360ef43c,28685,...,441.16,8ab6855b9fe9b812cd03a480a25058a1,2.0,Recomendo,Foi entregue somente 1. Quero saber do outro p...,2018-01-26,2018-01-27 09:16:56,37175,ilicinea,MG


In [22]:
# Сохраняем итоговую таблицу

orders_full.to_csv(os.path.join(PROCESSED_PATH, 'orders_full.csv'), index=False)


**orders_full** содержит всю информацию: пользователь, заказ, товар, оплата, отзывы, продавец. Используем how='left' — сохраняем все строки с заказами, даже если что-то отсутствует (например, отзывов нет). Пропуски не трогаем пока — это задача следующего этапа .